In [0]:
# use following command to make colab connect google drive
#The maximum lifetime of a Colab VM is 12 hours.
!dpkg --configure -a
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

In [10]:
#load the json data
import os
import json
data_path = '/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/'
data_exist=os.path.exists(data_path)
paragraphs = list()
labels = list()
if data_exist:
  print('Data exist')
  file_names = os.listdir(data_path)
  for i in range(len(file_names)):
      print(file_names[i])
      file_path = data_path+file_names[i]
      print(file_path)
      with open(file_path,encoding="utf8", errors='ignore') as f:
        data = json.load(f)
        for item in data['text'].values():
          paragraphs.append(item)

        for item in data['sentiment'].values():
          if item =="Neutral":
            labels.append(0)
          elif item == "Positive":
            labels.append(1)
          else:
            labels.append(-1)

Data exist
Team10_NVIDIA.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team10_NVIDIA.json
Team3_Facebook.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team3_Facebook.json
Team6_Tesla.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team6_Tesla.json
Team5_Microsoft.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team5_Microsoft.json
Team12_Chevron.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team12_Chevron.json
Team2_Amazon.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team2_Amazon.json
Team7_Walmart.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team7_Walmart.json
Team11_Boeing.json
/c

In [11]:
# Split data into train and test
train_size = int(len(paragraphs) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(paragraphs) - train_size))

Train size: 1315
Test size: 329


In [0]:

train_paragraphs = paragraphs[:train_size]
train_label = labels[:train_size]

test_paragraphs = paragraphs[train_size:]
test_label = labels[train_size:]

In [0]:
# Use TPOT API
!pip install tpot

In [16]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25)

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: LogisticRegression(DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=9, min_samples_leaf=6, min_samples_split=6), C=1.0, dual=True, penalty=l2)
0.96


In [0]:
# Use H2O API to analyze sentiment 

In [17]:
!pip install h2o

    100% |████████████████████████████████| 120.9MB 283kB/s 
  Stored in directory: /root/.cache/pip/wheels/f4/21/59/e953dbf8206d5061ce1346e119dad9ffafd7f88011430682ba
Successfully built h2o


In [70]:
import h2o

import itertools
import os
import json
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils
h2o.init(max_mem_size="4G",enable_assertions = False)
# enable_assertions = 

Checking whether there is an H2O instance running at http://localhost:54321. connected.


--------------------------  ---------------------------------------------------
H2O cluster uptime:         1 hour 23 mins
H2O cluster timezone:       Etc/UTC
H2O data parsing timezone:  UTC
H2O cluster version:        3.22.1.6
H2O cluster version age:    9 days
H2O cluster name:           H2O_from_python_unknownUser_of71o9
H2O cluster total nodes:    1
H2O cluster free memory:    3.985 Gb
H2O cluster total cores:    2
H2O cluster allowed cores:  2
H2O cluster status:         locked, healthy
H2O connection url:         http://localhost:54321
H2O connection proxy:
H2O internal security:      False
H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, Core V4
Python version:             3.6.7 final
--------------------------  ---------------------------------------------------

In [21]:
#load the json data
data_path = '/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/'
data_exist=os.path.exists(data_path)
paragraphs = list()
labels = list()
if data_exist:
  print('Data exist')
  file_names = os.listdir(data_path)
  for i in range(len(file_names)):
      print(file_names[i])
      file_path = data_path+file_names[i]
      print(file_path)
      with open(file_path,encoding="utf8", errors='ignore') as f:
        data = json.load(f)
        for item in data['text'].values():
          paragraphs.append(item)

        for item in data['sentiment'].values():
          if item =="Neutral":
            labels.append(0)
          elif item == "Positive":
            labels.append(1)
          else:
            labels.append(-1)


Data exist
Team9_GoldmanSachs.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team9_GoldmanSachs.json
Team7_Walmart.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team7_Walmart.json
Team2_Amazon.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team2_Amazon.json
Team4_Netflix.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team4_Netflix.json
Team10_NVIDIA.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team10_NVIDIA.json
Team12_Chevron.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team12_Chevron.json
Team6_Tesla.json
/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/data/Team6_Tesla.json
Team3_Facebook.js

In [36]:
df =  pd.DataFrame({'Text':paragraphs,'Sentiment':labels})
given_types = {'Text': 'string', 'Sentiment': 'int'}
hf = h2o.H2OFrame(df,column_types=given_types)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [37]:
print(len(hf))

1644


In [0]:
# Tokenize Words in Review
# Set Stop Words
# The STOP WORDS we are importing are from the nltk package
import pandas as pd
import os

# Use local data file or download from GitHub
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words
  
  
# Break reviews into sequence of words
words = tokenize(hf["Text"])

In [41]:
words.head()

C1 
 
 
 goldman 
 sachs 
 group 
 inc 
 nyse 
 gs 
 earnings 
 conference 
 call 
 january

In [44]:
!wget -P /content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/  https://s3.amazonaws.com/tomk/h2o-world/megan/w2v.hex

--2019-03-22 20:10:55--  https://s3.amazonaws.com/tomk/h2o-world/megan/w2v.hex
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.176.69
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.176.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8192011 (7.8M) [binary/octet-stream]
Saving to: ‘/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/w2v.hex’

w2v.hex             100%[===================>]   7.81M  6.94MB/s    in 1.1s    

2019-03-22 20:11:01 (6.94 MB/s) - ‘/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/w2v.hex’ saved [8192011/8192011]



In [71]:
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

# This takes time to run - left commented out
w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
w2v_model.train(training_frame=words)

# Pre-trained model available on s3: https://s3.amazonaws.com/tomk/h2o-world/megan/w2v.hex
# w2v_model = h2o.load_model("/content/drive/ColabWorkSpaces/INFO7374/Assignment3-Uncovering-Sentiments-using-EDGAR-Datasets/w2v.hex")

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [0]:
# Calculate a vector for each review
sentiment_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [0]:
# Add aggregated word embeddings 
ext_sentiment = hf.cbind(sentiment_vecs)

In [74]:
# Split data into train and test
train_size = int(len(paragraphs) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(paragraphs) - train_size))
ext_train = ext_sentiment[:train_size]
ext_test = ext_sentiment[train_size:]

Train size: 1315
Test size: 329


In [0]:
# Train GBM Model to Predict
from h2o.estimators import H2OGradientBoostingEstimator
predictors = ['Text']
response = 'Sentiment'
predictors =predictors+ sentiment_vecs.names

gbm_embeddings = H2OGradientBoostingEstimator(stopping_metric = "AUC", stopping_tolerance = 0.001,
                                              stopping_rounds = 5, score_tree_interval = 10,
                                              model_id = "gbm_embeddings.hex"
                                             )
gbm_embeddings.train(x = predictors, y = response, training_frame = ext_train, validation_frame = ext_test)